# Part 0: Data loading + basic hygiene

- We loaded `day.csv` into a dataframe and checked shape, column names, dtypes, missing values, summary stats.
- The dataset is clean; no meaningful missing values (confirmed via `isna().sum()`).
- Variables include weather (`temp`, `hum`, `windspeed`, `weathersit`), calendar (`season`, `yr`, `weekday`, `workingday`, `holiday`), and outcomes (`cnt`, `casual`, `registered`)
- We dropped `casual` and `registered` because `cnt = casual + registered` --> perfect leakage.
- We also dropped `atemp` because it's highly redundant with `temp`

### Choice of dataset (`day.csv` instead of `hour.csv`)

* We used **`day.csv`** to avoid strong temporal dependence present in the hourly data (e.g., autocorrelation across adjacent hours), which would violate basic linear model assumptions without more complex time-series handling.
* The daily dataset provides a **cleaner, more interpretable modeling setup** aligned with the assignment’s focus on regression diagnostics and interpretation rather than high-frequency temporal structure.

In [1]:
import pandas as pd
import numpy as np

# load day.csv 
df = pd.read_csv('day.csv')
df = df.drop(columns=['atemp', 'casual', 'registered'])
df['log_cnt'] = np.log(df['cnt'] + 1)

print(f"Dimensions: {df.shape}")
print(f"Columns: {list(df.columns)}\n")
df.info()
df.isna().sum()
df.describe()

categorical_cols = [ 'season', 'yr', 'mnth', 'weekday', 'weathersit', 'holiday', 'workingday' ]
numeric_cols = [ 'temp', 'hum', 'windspeed' ]

Dimensions: (731, 14)
Columns: ['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'hum', 'windspeed', 'cnt', 'log_cnt']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    int64  
 3   yr          731 non-null    int64  
 4   mnth        731 non-null    int64  
 5   holiday     731 non-null    int64  
 6   weekday     731 non-null    int64  
 7   workingday  731 non-null    int64  
 8   weathersit  731 non-null    int64  
 9   temp        731 non-null    float64
 10  hum         731 non-null    float64
 11  windspeed   731 non-null    float64
 12  cnt         731 non-null    int64  
 13  log_cnt     731 non-null    float64
dtypes: float64(4), int64(9), object(1)
memory usage: 80.1+ KB


# Part 1: Baseline Linear Model and Interpretation

### 1) Baseline model specification

- We fit an initial multiple linear regression model with `cnt` as the response.
- Predictors were chosen based on subject-matter relevance rather than including all available variables.
- Weather variables (`temp`, `hum`, `windspeed`, `weathersit`) were included because weather conditions directly affect bike usage.
- Calendar variables (`season`, `yr`, `workingday`, `holiday`) were included to capture seasonal patterns, long-term trends, and differences between workdays and non-workdays.
- We avoided including redundant or overlapping variables (e.g., month in addition to season) to keep the baseline model interpretable.

### 2) Coefficient estimates and interpretation

- We reported coefficient estimates, standard errors, and p-values from the fitted OLS model.
- Temperature (`temp`) has a positive coefficient, indicating higher rentals on warmer days, holding other factors fixed.
- Humidity (`hum`) and windspeed (`windspeed`) have negative coefficients, indicating reduced rentals under less comfortable weather conditions.
- Weather situation indicators (`weathersit`) show fewer rentals under worse weather categories relative to clear conditions.
- Seasonal indicators (`season`) show meaningful differences in rental levels across seasons.
- The year indicator (`yr`) has a positive coefficient, reflecting higher demand in the second year of the dataset.

### 3) Statistical significance vs practical relevance

- Many predictors are statistically significant due to the relatively large sample size.
- Practical relevance was assessed by the magnitude of coefficients rather than p-values alone.
- Weather and seasonal effects are large enough to be meaningful in practice, while some calendar effects are smaller despite statistical significance.

In [2]:
import statsmodels.formula.api as smf

model = smf.ols(formula="cnt ~ temp + hum + windspeed + C(weathersit) + C(season) + C(yr) + C(weekday) + workingday + holiday", data=df).fit()
print(model.summary())

coef_table = pd.DataFrame({
    "coef": model.params,
    "std_err": model.bse,
    "p_value": model.pvalues
}).sort_values("p_value")

coef_table_rounded = coef_table.copy()
coef_table_rounded["coef"] = coef_table_rounded["coef"].round(3)
coef_table_rounded["std_err"] = coef_table_rounded["std_err"].round(3)
coef_table_rounded["p_value"] = coef_table_rounded["p_value"].apply(lambda x: f"{x:.3g}")
coef_table_rounded


                            OLS Regression Results                            
Dep. Variable:                    cnt   R-squared:                       0.827
Model:                            OLS   Adj. R-squared:                  0.824
Method:                 Least Squares   F-statistic:                     213.9
Date:                Sun, 01 Feb 2026   Prob (F-statistic):          1.87e-259
Time:                        18:26:24   Log-Likelihood:                -5927.6
No. Observations:                 731   AIC:                         1.189e+04
Df Residuals:                     714   BIC:                         1.197e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           1313.5531    233

,coef,std_err,p_value
C(yr)[T.1],2017.136,61.139,1.01e-145
temp,5090.745,304.038,2.49e-53
C(season)[T.4],1543.868,95.720,3.91e-50
C(season)[T.2],1139.255,112.437,1.23e-22
C(weathersit)[T.3],-1976.186,206.310,1.58e-20
windspeed,-2779.818,424.021,1.06e-10
C(season)[T.3],842.925,148.503,2e-08
Intercept,1313.553,233.542,2.67e-08
C(weathersit)[T.2],-450.895,80.706,3.29e-08
workingday,335.863,70.468,2.28e-06


In [3]:
log_model = smf.ols(formula="log_cnt ~ temp + hum + windspeed + C(weathersit) + C(season) + C(yr) + C(weekday) + workingday + holiday", data=df).fit()
print(log_model.summary())

coef_table_log = pd.DataFrame({
    "coef": log_model.params,
    "std_err": log_model.bse,
    "p_value": log_model.pvalues
}).sort_values("p_value")

coef_table_log_rounded = coef_table_log.copy()
coef_table_log_rounded["coef"] = coef_table_log_rounded["coef"].round(3)
coef_table_log_rounded["std_err"] = coef_table_log_rounded["std_err"].round(3)
coef_table_log_rounded["p_value"] = coef_table_log_rounded["p_value"].apply(lambda x: f"{x:.3g}")
coef_table_log_rounded

                            OLS Regression Results                            
Dep. Variable:                log_cnt   R-squared:                       0.747
Model:                            OLS   Adj. R-squared:                  0.742
Method:                 Least Squares   F-statistic:                     132.0
Date:                Sun, 01 Feb 2026   Prob (F-statistic):          1.10e-200
Time:                        18:26:24   Log-Likelihood:                -139.04
No. Observations:                 731   AIC:                             312.1
Df Residuals:                     714   BIC:                             390.2
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              7.4215      0

,coef,std_err,p_value
Intercept,7.422,0.085,0
C(yr)[T.1],0.465,0.022,3.48e-76
C(season)[T.4],0.489,0.035,8.82e-40
temp,1.509,0.111,8.01e-38
C(weathersit)[T.3],-0.968,0.075,2.4e-34
C(season)[T.2],0.353,0.041,4.29e-17
windspeed,-0.842,0.154,6.72e-08
C(season)[T.3],0.235,0.054,1.5e-05
workingday,0.100,0.026,0.000101
hum,-0.380,0.107,0.0004


# Part 2: Transformations and Diagnostics

### 3. Assess linear model assumptions using diagnostics

* Residuals vs fitted values show clear curvature and increasing spread at higher fitted values, indicating nonlinearity and heteroskedasticity.
* Residuals vs key predictors (`temp`, `hum`) exhibit systematic patterns rather than random scatter.
* The Q–Q plot of residuals shows strong deviations in the tails, indicating non-normal errors.
* Residual variance increases with fitted values, violating the constant variance assumption.
* Leverage and Cook’s distance plots identify a small number of influential observations, though none dominate the fit.

### 4. Transformation strategy

* Due to skewness, heteroskedasticity, and non-normal residuals, we applied a log transformation to the response: `log(cnt + 1)`.
* The log transformation stabilizes variance and reduces the influence of extreme high-count days.
* No predictor transformations or interaction terms were added at this stage to keep interpretation simple.
* The transformed model shows visibly improved residual behavior in diagnostic plots.

### 5. Compare baseline and transformed models

* In-sample fit is comparable, with slightly lower adjusted ( R^2 ) after transformation due to scale change.
* Cross-validated RMSE improves substantially on the log scale, indicating better predictive stability.
* Back-transformed RMSE reflects the difficulty of predicting large counts but remains competitive.
* Coefficients in the transformed model are interpreted multiplicatively (approximate percent changes), improving interpretability for relative effects.

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import os

def plot(input_model): 
    model_name = "model" if input_model == model else "log_model"

    fitted_vals = input_model.fittedvalues
    residuals = input_model.resid
    standardized_residuals = input_model.get_influence().resid_studentized_internal
    leverage = input_model.get_influence().hat_matrix_diag

    os.makedirs("./figures", exist_ok=True)

    # Residuals vs Fitted
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=fitted_vals, y=residuals)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel('Fitted Values')
    plt.ylabel('Residuals')
    plt.title('Residuals vs Fitted Values')
    plt.savefig(f"./figures/{model_name}_residuals_vs_fitted.png")
    plt.close()

    # Residuals vs Temp
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=df['temp'], y=residuals)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel('Temperature')
    plt.ylabel('Residuals')
    plt.title('Residuals vs Temperature')
    plt.savefig(f"./figures/{model_name}_residuals_vs_temp.png")
    plt.close()

    # Residuals vs Humidity
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=df['hum'], y=residuals)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel('Humidity')
    plt.ylabel('Residuals')
    plt.title('Residuals vs Humidity')
    plt.savefig(f"./figures/{model_name}_residuals_vs_humidity.png")
    plt.close()

    # Q-Q Plot
    plt.figure(figsize=(10, 6))
    sm.qqplot(standardized_residuals, line ='45', fit=True)
    plt.title('Q-Q Plot of Standardized Residuals')
    plt.savefig(f"./figures/{model_name}_qq_plot.png")
    plt.close()

    # Residuals vs Leverage
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=leverage, y=standardized_residuals)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel('Leverage')
    plt.ylabel('Standardized Residuals')
    plt.title('Standardized Residuals vs Leverage')
    plt.savefig(f"./figures/{model_name}_standardized_residuals_vs_leverage.png")
    plt.close()

    # Cook's Distance
    influence = model.get_influence()
    cooks_d = influence.cooks_distance[0]
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=np.arange(len(cooks_d)), y=cooks_d)
    plt.xlabel('Observation Index')
    plt.ylabel("Cook's Distance")
    plt.title("Cook's Distance for Each Observation")
    plt.savefig(f"./figures/{model_name}_cooks_distance.png")
    plt.close()

plot(model)
plot(log_model)

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

# Part 3: Collinearity

### 6. Evaluate collinearity among predictors

* We computed a correlation matrix for numeric predictors (`temp`, `hum`, `windspeed`); all pairwise correlations are weak to moderate (|r| ≤ ~0.25).
* No strong linear dependence is observed among numeric predictors.
* We computed variance inflation factors (VIFs) for all predictors in the baseline model.
* Numeric predictors have acceptable VIFs (`temp` ≈ 3.4, `hum` ≈ 1.9, `windspeed` ≈ 1.2).
* Several categorical indicators (`weekday`, `workingday`, `holiday`) show infinite VIFs due to redundancy in dummy encoding.

### 7. Identify strongly collinear predictors and discuss implications

* No substantively strong collinearity exists among the main numeric predictors.
* Collinearity arises primarily from overlapping calendar variables (`weekday`, `workingday`, `holiday`).
* This redundancy inflates standard errors and makes individual weekday effects difficult to interpret.
* Predictive performance is largely unaffected, as shown by stable cross-validated RMSE.
* Collinearity can be addressed by dropping redundant categorical variables or using regularization methods.

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

corr_matrix = df[numeric_cols].corr()
plt.Figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Numeric Predictors')
plt.savefig("./figures/correlation_matrix.png")
plt.close()

X = log_model.model.exog
X_names = log_model.model.exog_names
vif_data = pd.DataFrame({
    "Variable": X_names,
    "VIF": [variance_inflation_factor(X, i) for i in range(X.shape[1])]
})
vif_data[vif_data["Variable"] != "Intercept"].sort_values("VIF", ascending=False)

vif_data

c:\Users\haesu\Desktop\Stuff\data210p_assignments\.venv\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


,Variable,VIF
0,Intercept,60.195265
1,C(weathersit)[T.2],1.608240
2,C(weathersit)[T.3],1.310736
3,C(season)[T.2],2.627952
4,C(season)[T.3],4.649735
5,C(season)[T.4],1.862739
6,C(yr)[T.1],1.031355
7,C(weekday)[T.1],inf
8,C(weekday)[T.2],inf
9,C(weekday)[T.3],inf


# Part 4: Model Selection and Validation

### 8) Cross-validation for out-of-sample performance

* We used K-fold cross-validation and chose RMSE as the performance metric.
* The baseline OLS model (on `cnt`) achieved a CV RMSE of 821.8 ± 86.6, indicating substantial prediction error and variability.
* The log-transformed OLS model (on `log_cnt`) achieved a much lower CV RMSE of 0.292 ± 0.101 on the log scale.
* When back-transforming predictions to the original scale, the log model had a CV RMSE of 4840.1 ± 278.6, reflecting the nonlinear error amplification introduced by exponentiation.
* Cross-validation confirms that the log transformation stabilizes variance and improves predictive behavior, even though scale-dependent RMSE comparisons must be interpreted carefully.

### 9) Model selection procedure

* We performed backward elimination starting from the full log-transformed OLS model.
* AIC was used as the selection criterion to balance goodness-of-fit and model complexity.
* Dropping `C(weekday)` reduced AIC from 312.09 → 310.39, indicating a modest improvement in model parsimony.
* The selected model retained weather, seasonal, year, and working-day effects while removing weak weekday indicators.
* This resulted in a simpler model with 11 predictors and minimal loss of explanatory power.

### 10) Comparison of selected vs baseline model

* In-sample fit was very similar: baseline Adj. R² = 0.7416, selected model Adj. R² = 0.7405.
* The selected model achieved a slightly lower AIC (310.39 vs 312.09), favoring it under information criteria.
* Cross-validated RMSE on the log scale remained essentially unchanged (0.290 ± 0.102), indicating no degradation in predictive performance.
* Removing `weekday` improved interpretability without sacrificing accuracy.
* Overall, the selected model is preferred due to comparable predictive performance with fewer, more meaningful predictors.

In [6]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def cv_rmse_ols(df, formula, response_col, k=10, seed=42, back_transform=False): 
    kf = KFold(n_splits=k, shuffle=True, random_state=seed)
    rmses = []

    for train_idx, test_idx in kf.split(df):
        train_data = df.iloc[train_idx]
        test_data = df.iloc[test_idx]

        model = smf.ols(formula=formula, data=train_data).fit()
        pred = model.predict(test_data)

        if back_transform: 
            pred = np.exp(pred) - 1

        rmse = np.sqrt(mean_squared_error(test_data[response_col], pred))
        rmses.append(rmse)

    return np.mean(rmses), np.std(rmses)

baseline_formula = "cnt ~ temp + hum + windspeed + C(weathersit) + C(season) + C(yr) + C(weekday) + workingday + holiday"
log_formula = "log_cnt ~ temp + hum + windspeed + C(weathersit) + C(season) + C(yr) + C(weekday) + workingday + holiday"

baseline_rmse_mean, baseline_rmse_std = cv_rmse_ols(df, baseline_formula, 'cnt')
log_rmse_mean, log_rmse_std = cv_rmse_ols(df, log_formula, 'log_cnt')
log_cnt_rmse_mean, log_cnt_rmse_std = cv_rmse_ols(df, log_formula, 'log_cnt', back_transform=True)

print(f"Baseline OLS CV RMSE: {baseline_rmse_mean:.3f} ± {baseline_rmse_std:.3f}")
print(f"Log-Transformed OLS CV RMSE: {log_rmse_mean:.3f} ± {log_rmse_std:.3f}")
print(f"Log-Transformed OLS CV RMSE (Back-Transformed): {log_cnt_rmse_mean:.3f} ± {log_cnt_rmse_std:.3f}")

Baseline OLS CV RMSE: 821.765 ± 86.563
Log-Transformed OLS CV RMSE: 0.292 ± 0.101
Log-Transformed OLS CV RMSE (Back-Transformed): 4840.139 ± 278.626


In [7]:
def backward_aic(df, formula):
    current_formula = formula
    current_model = smf.ols(current_formula, data=df).fit()
    current_aic = current_model.aic

    improved = True

    while improved:
        improved = False
        terms = current_model.model.formula.split('~')[1].strip().split('+')
        terms = [t.strip() for t in terms]

        aic_candidates = []

        for term in terms:
            if term == '1':
                continue
            reduced_terms = [t for t in terms if t != term]
            reduced_formula = "log_cnt ~ " + " + ".join(reduced_terms)
            reduced_model = smf.ols(reduced_formula, data=df).fit()
            aic_candidates.append((reduced_model.aic, reduced_formula, term))

        best_candidate = min(aic_candidates, key=lambda x: x[0])

        if best_candidate[0] < current_aic:
            current_aic, current_formula, dropped = best_candidate
            current_model = smf.ols(current_formula, data=df).fit()
            print(f"Dropped {dropped}, new AIC = {current_aic:.2f}")
            improved = True

    return current_model, current_formula

selected_model, selected_formula = backward_aic(df, log_formula)
print(selected_formula)
print(selected_model.summary())

baseline_model = smf.ols(log_formula, data=df).fit()

print("Baseline AIC:", baseline_model.aic)
print("Selected AIC:", selected_model.aic)

print("Baseline Adj R²:", baseline_model.rsquared_adj)
print("Selected Adj R²:", selected_model.rsquared_adj)

selected_rmse_mean, selected_rmse_std = cv_rmse_ols(
    df,
    formula=selected_formula,
    response_col="log_cnt",
    k=10
)

print(f"Selected model CV RMSE (log scale): {selected_rmse_mean:.3f} ± {selected_rmse_std:.3f}")

Dropped C(weekday), new AIC = 310.39
log_cnt ~ temp + hum + windspeed + C(weathersit) + C(season) + C(yr) + workingday + holiday
                            OLS Regression Results                            
Dep. Variable:                log_cnt   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     190.4
Date:                Sun, 01 Feb 2026   Prob (F-statistic):          1.68e-204
Time:                        18:26:27   Log-Likelihood:                -143.19
No. Observations:                 731   AIC:                             310.4
Df Residuals:                     719   BIC:                             365.5
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0

# Part 5: Ridge and Lasso Regression

### 11. Fit ridge and lasso with CV-selected tuning parameter (λ)

* We fit Ridge and Lasso regression models on the same predictor set as the selected log-OLS model (response = `log_cnt`).
* We used cross-validation to choose the regularization strength (reported as `alpha`, which corresponds to λ).

### 12. Report selected λ values and cross-validated performance

* Ridge: `alpha = 3.4891`, `CV RMSE ≈ 0.337` (log scale).
* Lasso: `alpha = 0.0001`, `CV RMSE ≈ 0.337` (log scale).
* Lasso selected 8 / 8 features, meaning it did not drop any predictors in this setup.
* The near-zero Lasso alpha suggests the data did not strongly favor sparse feature selection.

### 13. Compare ridge, lasso, and selected OLS model

* Predictive accuracy: Ridge and Lasso achieved similar RMSE (~0.337) and did not clearly outperform the selected log-OLS model (CV RMSE ~0.290).
* Interpretability: OLS is easiest to interpret directly; Ridge shrinks coefficients (less direct magnitude interpretation), and Lasso only improves interpretability if it sets coefficients to zero (it did not here).
* Effect on collinearity: Ridge reduces coefficient instability by shrinking correlated predictors together; Lasso can pick one of a correlated group, but in this case it kept all features.
* Overall, regularization did not provide a clear benefit here because the selected OLS model was already fairly simple and collinearity was manageable after removing `weekday`.

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict

num_cols = ["temp", "hum", "windspeed"]
cat_cols = ["weathersit", "season", "yr"]
bin_cols = ["workingday", "holiday"]

X = df[num_cols + cat_cols + bin_cols]
y = df["log_cnt"]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", "passthrough", cat_cols),
        ("bin", "passthrough", bin_cols)
    ], 
    remainder='drop'
)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

alphas = np.logspace(-4, 4, 200)

In [9]:
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.pipeline import Pipeline

ridge = Pipeline(steps=[
    ("prep", preprocess),
    ("model", RidgeCV(alphas=alphas, cv=kf, scoring="neg_root_mean_squared_error"))
])

ridge.fit(X, y)
ridge_alpha = ridge.named_steps["model"].alpha_

lasso = Pipeline(steps=[
    ("prep", preprocess),
    ("model", LassoCV(alphas=alphas, cv=kf, random_state=42, max_iter=20000))
])

lasso.fit(X, y)
lasso_alpha = lasso.named_steps["model"].alpha_

def cv_rmse_pipeline(pipe, X, y, cv):
    preds = cross_val_predict(pipe, X, y, cv=cv)
    rmse = np.sqrt(mean_squared_error(y, preds))
    return rmse

ridge_rmse = cv_rmse_pipeline(ridge, X, y, kf)
lasso_rmse = cv_rmse_pipeline(lasso, X, y, kf)

print(f"Ridge alpha: {ridge_alpha:.4f}, RMSE: {ridge_rmse:.3f}")
print(f"Lasso alpha: {lasso_alpha:.4f}, RMSE: {lasso_rmse:.3f}")

Ridge alpha: 3.4891, RMSE: 0.337
Lasso alpha: 0.0001, RMSE: 0.337


In [10]:
def cv_rmse_pipeline(pipe, X, y, cv):
    preds = cross_val_predict(pipe, X, y, cv=cv)
    rmse = np.sqrt(mean_squared_error(y, preds))
    return rmse

ridge_rmse = cv_rmse_pipeline(ridge, X, y, kf)
lasso_rmse = cv_rmse_pipeline(lasso, X, y, kf)

ridge_rmse, lasso_rmse

# Get feature names after preprocessing
ohe = lasso.named_steps["prep"].named_transformers_["cat"]
cat_feature_names = ohe.get_feature_names_out(cat_cols)

feature_names = (
    num_cols
    + list(cat_feature_names)
    + bin_cols
)

lasso_coefs = lasso.named_steps["model"].coef_
nonzero = np.sum(lasso_coefs != 0)

print(f"Lasso selected {nonzero} features out of {len(feature_names)}")

Lasso selected 8 features out of 8
